# Week 1 – Build a Better Fishing Net
### Interactive exploration of grids & connectivity

> **Instructions**  
> 1. **Duplicate** this notebook (File ▸ Save a Copy) so your edits don’t change the master.  
> 2. Run cells top‑to‑bottom.  
> 3. Click strands to cut/restore them – watch the status update.


## Why nets matter
If you cut too many ropes in a fishing net, fish slip through the gap and escape.  
Today you’ll investigate **how a grid‑shaped net breaks apart** and discover some graph theory along the way.

In [ ]:
# Interactive backend (needs the ipympl / jupyter-matplotlib package)
import ipympl
%matplotlib

import matplotlib.pyplot as plt
import ipywidgets as widgets


In [ ]:
def draw_intact_net(rows=5, cols=7, lw=2):
    """Draws a fresh rows×cols grid‑net."""
    plt.figure(figsize=(cols, rows))
    for r in range(rows):
        for c in range(cols):
            if c < cols - 1:                # horizontal
                plt.plot([c, c + 1], [rows - r]*2, 'k-', linewidth=lw)
            if r < rows - 1:                # vertical
                plt.plot([c]*2, [rows - 1 - r, rows - r], 'k-', linewidth=lw)
    plt.axis('off')
    plt.title('A brand‑new fishing net – no cuts yet!')
    plt.show()


In [ ]:
draw_intact_net()

### Predict first
If you cut **one** strand, can the net split? How few cuts *guarantee* a split?  
Write your guess here before experimenting:

`Your prediction:`


In [ ]:
# ----------------------------------------------------------
# Click‑to‑Cut Net Explorer  (interactive)
# ----------------------------------------------------------
ROWS, COLS = 5, 7         # tweak for larger/smaller nets

# Build list of all strands
def build_edges(rows, cols):
    e = []
    for r in range(rows):
        for c in range(cols):
            if c < cols - 1: e.append(((r,c), (r,  c+1)))  # horizontal
            if r < rows - 1: e.append(((r,c), (r+1,c)))    # vertical
    return e

edges = build_edges(ROWS, COLS)
cut_edges = set()          # user‑selected cuts
edge2line = {}

def is_connected(kept):
    if not kept: return False
    graph = {}
    for a,b in kept:
        graph.setdefault(a,[]).append(b)
        graph.setdefault(b,[]).append(a)
    start = next(iter(graph))
    seen, stack = set(), [start]
    while stack:
        node = stack.pop()
        if node in seen: continue
        seen.add(node)
        stack.extend(graph[node])
    return len(seen) == ROWS * COLS

# Create figure
fig, ax = plt.subplots(figsize=(COLS, ROWS))
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
ax.set_xlim(-0.5, COLS + 0.5)
ax.set_ylim(-0.5, ROWS + 0.5)

def draw_grid():
    edge2line.clear()
    ax.clear()
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_xlim(-0.5, COLS + 0.5)
    ax.set_ylim(-0.5, ROWS + 0.5)
    for edge in edges:
        (r1,c1),(r2,c2) = edge
        x = [c1, c2]
        y = [ROWS - r1, ROWS - r2]      # flip y so row0 is bottom
        intact = edge not in cut_edges
        color = 'k' if intact else 'r'
        style = '-' if intact else '--'
        line, = ax.plot(x, y, color=color, linestyle=style,
                        linewidth=2)
        line.set_picker(True)
        line.set_pickradius(5)          # 5 px tolerance
        edge2line[edge] = line
    fig.canvas.draw_idle()

# Status widget
status = widgets.HTML()

def update_status():
    kept = [e for e in edges if e not in cut_edges]
    intact = is_connected(kept)
    txt = f"<b>Cuts:</b> {len(cut_edges)}  <b>Net intact?</b> "
    txt += "✅ YES" if intact else "<span style='color:red;font-weight:bold;'>❌ NO – It split!</span>"
    status.value = txt

def on_pick(event):
    # Identify clicked edge
    clicked_line = event.artist
    edge = next(e for e,l in edge2line.items() if l is clicked_line)
    if edge in cut_edges:
        cut_edges.remove(edge)
    else:
        cut_edges.add(edge)
    draw_grid()
    update_status()

fig.canvas.mpl_connect('pick_event', on_pick)

# Reset button
reset_btn = widgets.Button(description='Reset net', button_style='info')
def _reset(b):
    cut_edges.clear()
    draw_grid()
    update_status()
reset_btn.on_click(_reset)

draw_grid()
update_status()
display(widgets.VBox([status, reset_btn]))


## Challenge Problems
1. **Tipping point** – How many strands can you cut while the net stays intact?  
2. **Certain split** – What’s the smallest number of strands that *guarantees* a split, no matter which you pick?  
3. **Scale test** – Change `ROWS, COLS` and look for patterns.


## Real‑World Extension
Build a mini‑net from string (e.g., 3 × 4). Cut strands physically and compare when it splits with your simulation.

### Reflection Journal
- Which mistake taught you the most?  
- Describe an extreme case that clarified your thinking.
